# Ch4: Manipulating Tensors
Optimizing tensor operations is very important when dealing with large amounts of complex data.

In [1]:
import torch
print(torch.__version__)

2.6.0+cu124


## Tensor Operations

### Indexing & Slicing
Indexing and slicing are the exact same as NumPy `ndarray`s. If we want to get the Python value, we call `tensor.item()`.

In [2]:
one_dim_tensor = torch.tensor([1,2,3,4,5,6,7,8])
print(one_dim_tensor[2])
print(one_dim_tensor[2].item())

tensor(3)
3


Slicing follows the following syntax: `[start:end:step]` (`start` inclusive, `end` exclusive)

In [3]:
one_dim_tensor[1:4]

tensor([2, 3, 4])

Here's a 2D example.

In [4]:
two_dim_tensor = torch.tensor([
    [1,2,3,4,5,6],
    [7,8,9,10,11,12],
    [13,14,15,16,17,18],
    [19,20,21,22,23,24]
])

two_dim_tensor[1][3]

tensor(10)

In [5]:
print("first 3 elements of 1st row: ", two_dim_tensor[0, 0:3])
print("first 4 elements of 2nd row: ", two_dim_tensor[1,0:4])

first 3 elements of 1st row:  tensor([1, 2, 3])
first 4 elements of 2nd row:  tensor([ 7,  8,  9, 10])


We can also index with criteria.

In [6]:
two_dim_tensor[two_dim_tensor<11]

tensor([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10])

### Combining & Splitting

To combine tensors, we can use `torch.stack()`.

In [7]:
torch.stack((two_dim_tensor, two_dim_tensor))

tensor([[[ 1,  2,  3,  4,  5,  6],
         [ 7,  8,  9, 10, 11, 12],
         [13, 14, 15, 16, 17, 18],
         [19, 20, 21, 22, 23, 24]],

        [[ 1,  2,  3,  4,  5,  6],
         [ 7,  8,  9, 10, 11, 12],
         [13, 14, 15, 16, 17, 18],
         [19, 20, 21, 22, 23, 24]]])

To split tensors, we use `torch.unbind()`.

In [8]:
torch.unbind(two_dim_tensor)

(tensor([1, 2, 3, 4, 5, 6]),
 tensor([ 7,  8,  9, 10, 11, 12]),
 tensor([13, 14, 15, 16, 17, 18]),
 tensor([19, 20, 21, 22, 23, 24]))

By default, PyTorch splits by row. We can use `dim=1` to split by column instead.

In [9]:
torch.unbind(two_dim_tensor,dim=1)

(tensor([ 1,  7, 13, 19]),
 tensor([ 2,  8, 14, 20]),
 tensor([ 3,  9, 15, 21]),
 tensor([ 4, 10, 16, 22]),
 tensor([ 5, 11, 17, 23]),
 tensor([ 6, 12, 18, 24]))

## Math Functions
PyTorch has a lot of built-in math expressions.

**Pointwise operations** (aka element-wise operations) perform on each individual element and then return the new tensor.
- Math functions
- Truncation
- Logical functions
- Trig functions

**Reduction operations** reduces the dimensionality or rank of a tensor.
- Statistical functions like mean or mode

**Comparison functions** compares values within a tensor or between 2 tensors
- Min/max values
- Sort values
- Test tensor status or conditions

**Linear algebra** enables matrix opeations
- Essential for deep-learning computations
- Matrix and tensor computations


**Spectral** and other math functions are useful for data transformation and analysis

### Basic Pointwise Functions

In [10]:
a = torch.tensor([10,2,8,6,4])
b = torch.tensor([1,2,4,3,1])
print('a+b:', a.add(b))
print('a*b', a.mul(b))
print('a/b:', a.div(b))

a+b: tensor([11,  4, 12,  9,  5])
a*b tensor([10,  4, 32, 18,  4])
a/b: tensor([10.,  1.,  2.,  2.,  4.])


### Basic Reduction Functions
Note that we have to use floating point numbers because `mean()` and `std()` only work with floating points.

In [11]:
c = torch.tensor([[20, 14, 11, 8], [3, 19, 14, 6]], dtype=torch.float)
print('Mean:', torch.mean(c))
print('Median:', torch.median(c))
print('Mode:', torch.mode(c))
print('Std:', torch.std(c))

Mean: tensor(11.8750)
Median: tensor(11.)
Mode: torch.return_types.mode(
values=tensor([8., 3.]),
indices=tensor([3, 0]))
Std: tensor(6.0341)


## Linear Algebra
PyTorch has a module called `torch.linalg` that contains lots of useful linear algebra functions. These functions are the same as NumPy, but with the added functionalities of tensors.

We can use `torch.matmul()` for multiplication between any dimension tensors. 2 vectors will result in the scalar dot product.

In [12]:
# dot product
first_tensor = torch.tensor([1,2,3])
second_tensor = torch.tensor([4,5,6])

dot_product = torch.matmul(first_tensor,second_tensor)
dot_product

tensor(32)

In [13]:
# matrix multiplication
first_2d_tensor = torch.tensor([[1,2,3],[-1,-2,-3]])
second_2d_tensor = torch.tensor([[-1,-2],[4,5],[4,5]])

result_2d_tensor = torch.matmul(first_2d_tensor, second_2d_tensor)
result_2d_tensor

tensor([[ 19,  23],
        [-19, -23]])

We can also use `torch.mm()` which is a lot faster, but it only supports 2D tensors and doesn't support broadcasting.

`multi_dot()` can chain multiple matrix multiplications together, which is very useful for deep learning NNs.

In [14]:
# chained matrix multiplication
first_ten = torch.randn(2, 3)
second_ten = torch.randn(3, 4)
third_ten = torch.randn(4, 5)
fourth_ten = torch.randn(5, 6)
fifth_ten = torch.randn(6, 7)

torch.linalg.multi_dot((first_ten,second_ten,third_ten,fourth_ten,fifth_ten))

tensor([[-31.8836,   6.7960,   5.1101, -11.6770,  15.1170,  -8.0794,  -4.2135],
        [ -5.6098,  -7.4835,   9.1502,  16.5956, -14.7858,  -2.0040,  11.1746]])

## Automatic Differentiation (Autograd)
PyTorch automatically performs automatic differentiation for all tensor operations. This is very helpful for back propogation. Furthermore, we can view the individual gradients with the `grad` attribute.

In [19]:
x = torch.autograd.Variable(torch.Tensor([2]), requires_grad=True)
y = torch.autograd.Variable(torch.Tensor([1]), requires_grad=True)
z = torch.autograd.Variable(torch.Tensor([5]), requires_grad=True)

# forward prop
a = x - y
f = z * a
print(f'f = {f}')

# backward prop
f.backward()

print('Gradient value for x:', x.grad)
print('Gradient value for y:', y.grad)
print('Gradient value for z:', z.grad)

f = tensor([5.], grad_fn=<MulBackward0>)
Gradient value for x: tensor([5.])
Gradient value for y: tensor([-5.])
Gradient value for z: tensor([1.])
